In [1]:
#libraries
import cv2
import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean
import time

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


can I start from specific timestamps ?

what happens for skip_frames=0 ?

In [2]:
#read video file frame by frame
def read_video(video,skip_frames,resolution_width):
    resolution_height=round(int(resolution_width * 9/16))
    resolution=(resolution_width,resolution_height)
    vid = cv2.VideoCapture(video)
    frames=[]
    vid_length=0
    while(vid.isOpened()):
        # Capture frame-by-frame
        ret, frame = vid.read() # if ret is false, frame has no content
        if not ret:
            break
        if vid_length%skip_frames==0: # skip every "skip_frame"
            frame=cv2.resize(frame,resolution) # resize the video to a different resolution
            frames.append(frame) #add the individual frames to a list
        vid_length+=1 #increase the vid_length counter
    vid.release()
    cv2.destroyAllWindows()
    return frames

Dominant Color Extractor

- add better comments

In [3]:
def extract_dominant_color(frame_list,bin_threshold=0.05,colors_to_return=5):
    print(str(len(frame_list))+' frames to process.')
    start=time.time()
    rgb_to_color=fn_rgb_to_color() #get the color dict 
    bins={} #bins dict for histograms 
    for rgb in rgb_to_color: #init the dict with zeros for every key
        bins[rgb_to_color[rgb]]=0
    rgb_list=[] #create a traverseable list of the rgb_values
    for rgb in rgb_to_color: #map the values of the dict to a list
        rgb_list.append(rgb)
    i = 0
    for image in frame_list: #traverse the video
        #flatten the image to 1d 
        img = image.reshape((image.shape[0] * image.shape[1], 3))     
        for pixel in img: # do nearest neighbour search on every pixel every color in the list
            bin_aux=[]
            #get the euclidean distance between the colors and the current pixel
            for rgb in rgb_list:
                bin_aux.append(euclidean(pixel,rgb))
            # get the index of the color,which has the smallest distance, in rgb_list
            min_pos = np.argmin(bin_aux)
            #increment the respective color 
            bins[rgb_to_color[rgb_list[min_pos]]]+=1
        i+=1
        end=time.time()
        print('Finished '+str(i)+',time: '+str(end-start))
    #create a dataframe, sorted descending by count
    bins_sorted=sorted(zip(list(bins.values()),list(bins.keys())),reverse=True)
    df=pd.DataFrame(bins_sorted,columns=['count','color'])
    df.set_index('color',inplace=True) #set the colors as the index of the dataframe
    norm_factor = len(frame_list)* np.shape(frame_list[0])[0] * np.shape(frame_list[0])[1]  #normalize the bins
    df=df/norm_factor 
    bin_threshold=bin_threshold/100
    df = df[df>bin_threshold].dropna() #kick bins from the dataframe with precentage lower than bin_threshold 
    return df.head(colors_to_return)#return the color_return highest bins, default 5, if less bins then
                                #color_return are there return all

In [4]:
def fn_rgb_to_color():
    colors={'darkred':(139,0,0),
    'firebrick':(178,34,34),
    'crimson':(220,20,60),
    'red':(255,0,0),
    'tomato':(255,99,71),
    'salmon':(250,128,114),
    'darkorange':(255,140,0),
    'gold':(255,215,0),
    'darkkhaki':(189,183,107),
    'yellow':(255,255,0),
    'darkolivegreen':(85,107,47),
    'olivedrab':(107,142,35),
    'greenyellow':(173,255,47),
    'darkgreen':(0,100,0),
    'aquamarine':(127,255,212),
    'steelblue':(70,130,180),
    'skyblue':(135,206,235),
    'darkblue':(0,0,139),
    'blue':(0,0,255),
    'royalblue':(65,105,225),
    'purple':(128,0,128),
    'violet':(238,130,238),
    'deeppink':(255,20,147),
    'pink':(255,192,203),
    'antiquewhite':(250,235,215),
    'saddlebrown':(139,69,19),
    'sandybrown':(244,164,96),
    'ivory':(255,255,240),
    'dimgrey':(105,105,105),
    'grey':(28,128,128),
    'silver':(192,192,192),
    'lightgrey':(211,211,211),
    'black':(0,0,0),
    'white':(255,255,255),
    'darkcyan':(0,139,139),
    'cyan':(0,255,255),
    'green':(0,128,0),
    'khaki':(240,230,140),
    'goldenrod':(218,165,32),
    'orange':(255,165,0),
    'coral':(255,127,80),
    'magenta':(255,0,255),
    'wheat':(245,222,179),
    'skin':(255,224,189),
    'purple4':(147,112,219)}
    rgb_to_color={}
    for color in colors:
        rgb_to_color[colors[color]]=color
    #purple4 is median purple
    #skin is caucasian
    return rgb_to_color

In [48]:
#read .json-file
import json
with open('/home/jacob/Downloads/ImCo___Colour_Range.json') as file:
    ground_truth=json.load(file)
ground_truth['annotations'][0]['content']

'white, black, sandybrown, lightgrey, dimgrey, antiquewhite'

In [33]:
#embed the color strings as unique numbers 
color_embedding = {}
rgb_to_color = fn_rgb_to_color()
i=0
for rgb in rgb_to_color:
    color_embedding[rgb_to_color[rgb]]=i
    i+=1

In [38]:
#use the embedding on the ground truth
g_t_all_embedded = []
for g_t in ground_truth['annotations']:
    g_t=g_t['content']
    g_t_embedded=[]
    for color in g_t.replace(" ","").split(','):
        g_t_embedded.append(color_embedding[color])
    g_t_all_embedded.append(g_t_embedded)
# g_t_all_embedded

[[33, 32, 26, 31, 28, 24],
 [24, 32, 33, 43, 26, 31, 4],
 [33, 31, 32, 43, 16],
 [27, 32, 31, 33, 43, 16],
 [27, 16, 43, 25, 9],
 [27, 32, 31, 33, 43],
 [27, 16, 43],
 [27, 28, 31, 32, 33, 43, 16],
 [27, 32, 31, 33, 43],
 [27, 16, 43, 25, 9],
 [27, 28, 31, 32, 33, 43, 16],
 [27, 28, 31, 32, 43, 16, 0],
 [32, 43, 27, 26, 28, 33],
 [32, 28, 27, 15, 43, 33],
 [32, 43, 27, 31, 7],
 [32, 43, 27, 26, 28, 33],
 [32, 43, 27, 33, 24, 7, 0],
 [32, 17, 43, 27, 28, 0, 33, 26],
 [32, 43, 27, 33, 24, 7, 0],
 [32, 43, 27, 26, 28, 33],
 [32, 27, 29, 43, 25, 17, 33],
 [32, 17, 43, 27, 28, 0, 33, 26],
 [32, 29, 28, 27, 43, 26, 25, 33, 0],
 [32, 43, 27, 33, 24, 0],
 [32, 28, 27, 15, 43, 33, 0],
 [32, 28, 27, 43, 33, 0],
 [32, 27, 29, 43, 25, 17, 33],
 [32, 43, 27, 26, 28, 33],
 [32, 27, 43, 33, 24, 0],
 [32, 29, 28, 27, 43, 26, 33, 0],
 [32, 27, 29, 43, 33, 24, 7],
 [32, 27, 25, 17, 15, 43, 33],
 [32, 28, 27, 43, 25, 26, 15, 33],
 [32, 43, 27, 26, 28, 15, 33],
 [32, 43, 27, 31, 7],
 [32, 28, 27, 43, 25, 

In [8]:
#precision & recall
from sklearn.metrics import recall_score,precision_score
recall_score(g_t_embedded,g_t_embedded,average='micro')
precision_score(g_t_embedded,g_t_embedded,average='micro')

1.0

In [9]:
gt_e_s_list =[]
for i,content in enumerate(ground_truth['annotations']):
    gt_e_s=[]
    gt_e_s.append(round(content['begin']/1000/60*25))
    gt_e_s.append(round(content['end']/1000/60*25))
    gt_e_s_list.append(gt_e_s)

read random segments of the video

In [10]:
#read video file frame by frame
def read_video_segments(video,start_frame,end_frame,resolution_width):
    resolution_height=round(int(resolution_width * 9/16))
    resolution=(resolution_width,resolution_height)
    vid = cv2.VideoCapture(video)
    frames=[]
    vid_length=0
    while(vid.isOpened()):
        # Capture frame-by-frame
        ret, frame = vid.read() # if ret is false, frame has no content
        if not ret:
            break
        # skip every "skip_frame"
        if vid_length>=start_frame:
            # resize the video to a different resolution
            frame=cv2.resize(frame,resolution)
            frames.append(frame) #add the individual frames to a list
        if vid_length==end_frame:
            break
        vid_length+=1 #increase the vid_length counter
    vid.release()
    cv2.destroyAllWindows()
    return frames

In [11]:
segment_list=[]
for gt_i in gt_e_s_list:
    segment_list.append(read_video_segments('/home/jacob/Downloads/Wells_John_CompanyMen_full.mp4',
                   gt_i[0],gt_i[1],200))

In [13]:
dominant_colors_list=[]
for segment in segment_list:
    dominant_colors_list.append(extract_dominant_color(segment))

10 frames to process.
Finished 1,time: 14.629685401916504
Finished 2,time: 28.81556463241577
Finished 3,time: 42.20822286605835
Finished 4,time: 55.57504439353943
Finished 5,time: 69.15574026107788
Finished 6,time: 82.70095252990723
Finished 7,time: 96.07172679901123
Finished 8,time: 109.60807466506958
Finished 9,time: 122.86536693572998
Finished 10,time: 136.13143658638
3 frames to process.
Finished 1,time: 14.294831991195679
Finished 2,time: 28.224335193634033
Finished 3,time: 42.215362787246704
4 frames to process.
Finished 1,time: 14.717067956924438
Finished 2,time: 29.766655921936035
Finished 3,time: 44.402613401412964
Finished 4,time: 58.730828523635864
4 frames to process.
Finished 1,time: 14.697574377059937
Finished 2,time: 29.43988823890686
Finished 3,time: 44.664732694625854
Finished 4,time: 59.231940269470215
2 frames to process.
Finished 1,time: 14.487981796264648
Finished 2,time: 29.219991207122803
1 frames to process.
Finished 1,time: 14.17618703842163
2 frames to process

In [37]:
pred_list=[]
for dominant_colors in dominant_colors_list:
    embedding=[]
    for color in dominant_colors.index.tolist():
        embedding.append(color_embedding[color])
    pred_list.append(embedding)
pred_list

[[17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 32, 18],
 [17, 32, 18],
 [17, 32, 18],
 [17, 32, 18],
 [17, 32, 18],
 [17, 32, 18],
 [17, 32, 18],
 [17, 32, 18],
 [32, 17, 18],
 [32, 17, 18],
 [32, 17, 18],
 [32, 17, 18],
 [32, 17, 18],
 [32, 17, 18],
 [32, 17, 18],
 [32, 17, 18],
 [32, 17, 18],
 [32, 17],
 [32, 17, 18],
 [32, 17, 18],
 [32, 17, 18],
 [32, 17, 18],
 [32, 17, 18],
 [32, 17, 18],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],

In [52]:
precision_score(g_t_all_embedded[6],pred_list[6],average='micro')

0.0

In [50]:
g_t_all_embedded
# ground_truth['annotations']

[[33, 32, 26, 31, 28, 24],
 [24, 32, 33, 43, 26, 31, 4],
 [33, 31, 32, 43, 16],
 [27, 32, 31, 33, 43, 16],
 [27, 16, 43, 25, 9],
 [27, 32, 31, 33, 43],
 [27, 16, 43],
 [27, 28, 31, 32, 33, 43, 16],
 [27, 32, 31, 33, 43],
 [27, 16, 43, 25, 9],
 [27, 28, 31, 32, 33, 43, 16],
 [27, 28, 31, 32, 43, 16, 0],
 [32, 43, 27, 26, 28, 33],
 [32, 28, 27, 15, 43, 33],
 [32, 43, 27, 31, 7],
 [32, 43, 27, 26, 28, 33],
 [32, 43, 27, 33, 24, 7, 0],
 [32, 17, 43, 27, 28, 0, 33, 26],
 [32, 43, 27, 33, 24, 7, 0],
 [32, 43, 27, 26, 28, 33],
 [32, 27, 29, 43, 25, 17, 33],
 [32, 17, 43, 27, 28, 0, 33, 26],
 [32, 29, 28, 27, 43, 26, 25, 33, 0],
 [32, 43, 27, 33, 24, 0],
 [32, 28, 27, 15, 43, 33, 0],
 [32, 28, 27, 43, 33, 0],
 [32, 27, 29, 43, 25, 17, 33],
 [32, 43, 27, 26, 28, 33],
 [32, 27, 43, 33, 24, 0],
 [32, 29, 28, 27, 43, 26, 33, 0],
 [32, 27, 29, 43, 33, 24, 7],
 [32, 27, 25, 17, 15, 43, 33],
 [32, 28, 27, 43, 25, 26, 15, 33],
 [32, 43, 27, 26, 28, 15, 33],
 [32, 43, 27, 31, 7],
 [32, 28, 27, 43, 25, 

In [49]:
pred_list
# dominant_colors_list

[[17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 18, 32],
 [17, 32, 18],
 [17, 32, 18],
 [17, 32, 18],
 [17, 32, 18],
 [17, 32, 18],
 [17, 32, 18],
 [17, 32, 18],
 [17, 32, 18],
 [32, 17, 18],
 [32, 17, 18],
 [32, 17, 18],
 [32, 17, 18],
 [32, 17, 18],
 [32, 17, 18],
 [32, 17, 18],
 [32, 17, 18],
 [32, 17, 18],
 [32, 17],
 [32, 17, 18],
 [32, 17, 18],
 [32, 17, 18],
 [32, 17, 18],
 [32, 17, 18],
 [32, 17, 18],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],
 [32, 18, 17],

In [45]:
color_embedding

{'antiquewhite': 24,
 'aquamarine': 14,
 'black': 32,
 'blue': 18,
 'coral': 40,
 'crimson': 2,
 'cyan': 35,
 'darkblue': 17,
 'darkcyan': 34,
 'darkgreen': 13,
 'darkkhaki': 8,
 'darkolivegreen': 10,
 'darkorange': 6,
 'darkred': 0,
 'deeppink': 22,
 'dimgrey': 28,
 'firebrick': 1,
 'gold': 7,
 'goldenrod': 38,
 'green': 36,
 'greenyellow': 12,
 'grey': 29,
 'ivory': 27,
 'khaki': 37,
 'lightgrey': 31,
 'magenta': 41,
 'olivedrab': 11,
 'orange': 39,
 'pink': 23,
 'purple': 20,
 'purple4': 44,
 'red': 3,
 'royalblue': 19,
 'saddlebrown': 25,
 'salmon': 5,
 'sandybrown': 26,
 'silver': 30,
 'skin': 43,
 'skyblue': 16,
 'steelblue': 15,
 'tomato': 4,
 'violet': 21,
 'wheat': 42,
 'white': 33,
 'yellow': 9}